# Spreadsheet - transpose 

### Transpose a file:
* **Select your spreadsheet file from the "Select File" dropdown listbox below.**
* **Press the "Transpose" button and the transposed spreadsheet will be written to a file by the same name of the input file with "_T" appended in the same directory of the input file.**

In [1]:
%%html
<style>
div.input {
    display:none;
}
div.output_stderr{
    display:none
}
</style>

In [2]:
#                                         single cell for select, open and transpose:
#                                         target directory set for docker run -v `pwd`:...   ==  mount user data
target_dir = './transform_data/'

import warnings
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd

from IPython.display import display
import ipywidgets as widgets

#                                         local function to open and transpose:
def transpose_selected_file(obie_jobie):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name = os.path.join(target_dir, flistbx.value)
    spreadsheet_df = pd.read_csv(file_name, sep='\t', index_col=0, header=0)
    spreadsheet_df = spreadsheet_df.transpose()
    name_base, file_extension = os.path.splitext(file_name)
    outfile_name = name_base + '_T.tsv'
    spreadsheet_df.to_csv(outfile_name, sep='\t')
    print('Output written to\n', outfile_name)

#                                         Get list of (docker run -v) mounted files:
flist = os.listdir(target_dir)
FEXT = ['.tsv', '.txt', '.df']
my_file_list = []
for f in flist:
    if os.path.isfile(os.path.join(target_dir, f)):
        noNeed, f_ext = os.path.splitext(f)
        if f_ext in FEXT:
            my_file_list.append(f)

#                                         (docker run -v) mounted files was empty:
if len(my_file_list) <= 0:
    my_file_list.append('No Data')

#                                         Create and display the widget controls:
flistbx = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select File:'
)
display(flistbx)

output_file_button = widgets.Button(
    description='Transpose',
    disabled=False,
    button_style='',
    tooltip='file to transpose',
    data_file_key='output_file_name'
    )
output_file_button.on_click(transpose_selected_file)
display(output_file_button)

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


# Spreadsheet - common samples 

### Return the subsets of two spreadsheets with only common samples of 2 spreadsheets: 
* **Select your two samples x phenotypes spreadsheet files respectively from the "Select File 1" and "Select File 2" dropdown listboxes below.**
* **Press the "Get Common Samples" button and the corresponding subsets of the spreadsheets with only common samples of 2 spreadsheets will be written to 2 files by the same names of 2 input files with "_Com" appended respectively in the same directory of the input files.**

In [3]:
%%html
<style>
div.input {
    display:none;
}
div.output_stderr{
    display:none
}
</style>

In [4]:
#                                         single cell for select, open and transpose:
#                                         target directory set for docker run -v `pwd`:...   ==  mount user data
target_dir = '../../'

import warnings
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd
import knpackage.toolbox as kn

from IPython.display import display
import ipywidgets as widgets

#                                         local function to read files and get common samples and write:
def get_common_samples(obie_jobie):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name_1 = os.path.join(target_dir, flistbx_1.value)
    file_name_2 = os.path.join(target_dir, flistbx_2.value)
    sxp_1_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    sxp_2_df = pd.read_csv(file_name_2, sep='\t', index_col=0, header=0)
    sxp_1_gene_names = kn.extract_spreadsheet_gene_names(sxp_1_df)
    sxp_2_gene_names = kn.extract_spreadsheet_gene_names(sxp_2_df)
    common_samples_list = kn.find_common_node_names(sxp_1_gene_names, sxp_2_gene_names)
    sxp_1_trim_df,sxp_2_trim_df = sxp_1_df.loc[common_samples_list], sxp_2_df.loc[common_samples_list]
    name_base_1, file_extension_1 = os.path.splitext(file_name_1)
    outfile_name_1 = name_base_1 + '_Com.tsv'
    name_base_2, file_extension_2 = os.path.splitext(file_name_2)
    outfile_name_2 = name_base_2 + '_Com.tsv'
    sxp_1_trim_df.to_csv(outfile_name_1, sep='\t', index=True, header=True)
    sxp_2_trim_df.to_csv(outfile_name_2, sep='\t', index=True, header=True)
    print('Outputs written to\n', outfile_name_1,'\nand\n',outfile_name_2)

#                                         Get list of (docker run -v) mounted files:
flist = os.listdir(target_dir)
FEXT = ['.tsv', '.txt', '.df']
my_file_list = []
for f in flist:
    if os.path.isfile(os.path.join(target_dir, f)):
        noNeed, f_ext = os.path.splitext(f)
        if f_ext in FEXT:
            my_file_list.append(f)

#                                         (docker run -v) mounted files was empty:
if len(my_file_list) <= 0:
    my_file_list.append('No Data')

#                                         Create and display the widget controls:
flistbx_1 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select File 1:'
)
display(flistbx_1)

flistbx_2 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select File 2:'
)
display(flistbx_2)

output_file_button = widgets.Button(
    description='Get Common Samples',
    disabled=False,
    button_style='',
    tooltip='get common samples button',
    data_file_key='output_file_name'
    )
output_file_button.on_click(get_common_samples)
display(output_file_button)


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


# Spreadsheet - merge 

### Combine two spreadsheets into one with all samples and phenotypes (NaN filled): 
* **Select your two samples x phenotypes spreadsheet files respectively from the "Select File 1" and "Select File 2" dropdown listboxes below.**
* **Press the "Merge" button and the union spreadsheet of those 2 spreadsheets will be written to a file by the name of first input file with the name of the second input file and "_Mrg" appended in the same directory of the input files.**

In [5]:
%%html
<style>
div.input {
    display:none;
}
div.output_stderr{
    display:none
}
</style>




In [6]:
#                                         single cell for select, open and transpose:
#                                         target directory set for docker run -v `pwd`:...   ==  mount user data
target_dir = '../../'

import warnings
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd
import knpackage.toolbox as kn

from IPython.display import display
import ipywidgets as widgets

#                                         local function to read files and get common samples and write:
def merge(obie_jobie):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name_1 = os.path.join(target_dir, flistbx_1.value)
    file_name_2 = os.path.join(target_dir, flistbx_2.value)

    spreadsheet_1_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    spreadsheet_2_df = pd.read_csv(file_name_2, sep='\t', index_col=0, header=0)
    spreadsheet_1_samples = kn.extract_spreadsheet_gene_names(spreadsheet_1_df)
    spreadsheet_2_samples = kn.extract_spreadsheet_gene_names(spreadsheet_2_df)
    
    #all_samples_list = kn.find_unique_node_names(spreadsheet_1_samples, spreadsheet_2_samples)
    
    spreadsheet_1_phenotypes = list(spreadsheet_1_df.columns)
    spreadsheet_2_phenotypes = list(spreadsheet_2_df.columns)
    
    #all_phenotypes_list = kn.find_unique_node_names(spreadsheet_1_phenotypes, spreadsheet_2_phenotypes)
    
    spreadsheet_X_df = pd.concat([spreadsheet_1_df, spreadsheet_2_df], axis=1)
    name_base_1, file_extension_1 = os.path.splitext(file_name_1)
    name_base_2, file_extension_2 = os.path.splitext(file_name_2)
    # print(os.path.basename(name_base_2))
    # print(os.path.relpath(name_base_2,start=target_dir))
    outfile_name = name_base_1 + '_' + os.path.basename(name_base_2) + '_Mrg.tsv'
    spreadsheet_X_df.to_csv(outfile_name, sep='\t', index=True, header=True)
    print('Output written to\n', outfile_name)

#                                         Get list of (docker run -v) mounted files:
flist = os.listdir(target_dir)
FEXT = ['.tsv', '.txt', '.df']
my_file_list = []
for f in flist:
    if os.path.isfile(os.path.join(target_dir, f)):
        noNeed, f_ext = os.path.splitext(f)
        if f_ext in FEXT:
            my_file_list.append(f)

#                                         (docker run -v) mounted files was empty:
if len(my_file_list) <= 0:
    my_file_list.append('No Data')

#                                         Create and display the widget controls:
flistbx_1 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select File 1:'
)
display(flistbx_1)

flistbx_2 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select File 2:'
)
display(flistbx_2)

output_file_button = widgets.Button(
    description='Merge',
    disabled=False,
    button_style='',
    tooltip='merge button',
    data_file_key='output_file_name'
    )
output_file_button.on_click(merge)
display(output_file_button)



Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


# Spreadsheet - select_genes 

### Return one spreadsheet with only those genes selected from an input list: 
* **Select your genes x samples spreadsheet file and gene list file respectively from the "Select Spreadsheet File" and "Select Gene List File" dropdown listboxes below.**
* **Press the "Select Genes" button and the spreadsheet with only those genes selected will be written to a file by same name of the input spreadsheet file with "_Slct_Gn" appended in the same directory of the input files.**

In [7]:
%%html
<style>
div.input {
    display:none;
}
div.output_stderr{
    display:none
}
</style>

In [8]:
#                                         single cell for select, open and transpose:
#                                         target directory set for docker run -v `pwd`:...   ==  mount user data
target_dir = '../../'

import warnings
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd
import knpackage.toolbox as kn

from IPython.display import display
import ipywidgets as widgets

# utility
def read_a_list_file(input_file_name):
    """
    Args:
        input_file_name:     full path name of a file containing a list
    Returns:
        a list that is contained in the file
    """
    with open(input_file_name, 'r') as fh:
        str_input = fh.read()
    return list(str_input.split())

#                                         local function to read files and get common samples and write:
def select_genes(obie_jobie):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name_1 = os.path.join(target_dir, flistbx_1.value)
    file_name_2 = os.path.join(target_dir, flistbx_2.value)

    gene_select_list = read_a_list_file(file_name_2)
    spreadsheet_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    gene_names = kn.extract_spreadsheet_gene_names(spreadsheet_df)
    intersection_names = kn.find_common_node_names(gene_names, gene_select_list)
    spreadsheet_intersected_df = spreadsheet_df.loc[intersection_names] 
    
    name_base_1, file_extension_1 = os.path.splitext(file_name_1)
    # print(os.path.basename(name_base_2))
    # print(os.path.relpath(name_base_2,start=target_dir))
    outfile_name = name_base_1 + '_Slt_Gn.tsv'
    spreadsheet_intersected_df.to_csv(outfile_name, sep='\t', index=True, header=True)
    print('Output written to\n', outfile_name)

#                                         Get list of (docker run -v) mounted files:
flist = os.listdir(target_dir)
FEXT = ['.tsv', '.txt', '.df']
my_file_list = []
for f in flist:
    if os.path.isfile(os.path.join(target_dir, f)):
        noNeed, f_ext = os.path.splitext(f)
        if f_ext in FEXT:
            my_file_list.append(f)

#                                         (docker run -v) mounted files was empty:
if len(my_file_list) <= 0:
    my_file_list.append('No Data')

#                                         Create and display the widget controls:
flistbx_1 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select Spreadsheet File:'
)
display(flistbx_1)

flistbx_2 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select Gene List File:'
)
display(flistbx_2)

output_file_button = widgets.Button(
    description='Select Genes',
    disabled=False,
    button_style='',
    tooltip='select genes button',
    data_file_key='output_file_name'
    )
output_file_button.on_click(select_genes)
display(output_file_button)


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


# Spreadsheet - cluster_averages 

### Return a spreadsheet of averages for each category given a genes x samples dataframe and a samples classification dictionary: 
* **Select your genes x samples spreadsheet file and samples classification dictionary file respectively from the "Select Spreadsheet File" and "Select Dictionary File" dropdown listboxes below.**
* **Press the "Get Cluster Averages" button the spreadsheet of averages for each category will be written to a file by the same name of the input spreadsheet file with "_Clst_Avg" appended in the same directory of the input files.**

In [9]:
%%html
<style>
div.input {
    display:none;
}
div.output_stderr{
    display:none
}
</style>

In [10]:
#                                         single cell for select, open and transpose:
#                                         target directory set for docker run -v `pwd`:...   ==  mount user data
target_dir = '../../'

import warnings
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd
import numpy as np

from IPython.display import display
import ipywidgets as widgets

#                                         local function to read files and get common samples and write:
def get_cluster_averages(obie_jobie):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name_1 = os.path.join(target_dir, flistbx_1.value)
    file_name_2 = os.path.join(target_dir, flistbx_2.value)

    spreadsheet_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    labels_df = pd.read_csv(file_name_2, sep='\t', index_col=0, names=['sample','cluster_number'])
    labels_dict = labels_df.to_dict()['cluster_number']
    cluster_numbers = list(np.unique(list(labels_dict.values())))
    labels = list(labels_dict.values())
    # labels == i is a boolean list
    cluster_ave_df = pd.DataFrame({i: spreadsheet_df.iloc[:, labels == i].mean(axis=1) for i in cluster_numbers})
    name_base_1, file_extension_1 = os.path.splitext(file_name_1)
    # print(os.path.basename(name_base_2))
    # print(os.path.relpath(name_base_2,start=target_dir))
    outfile_name = name_base_1 + '_Clst_Avg.tsv'
    cluster_ave_df.to_csv(outfile_name, sep='\t', index=True, header=True)
    print('Output written to\n', outfile_name)

#                                         Get list of (docker run -v) mounted files:
flist = os.listdir(target_dir)
FEXT = ['.tsv', '.txt', '.df']
my_file_list = []
for f in flist:
    if os.path.isfile(os.path.join(target_dir, f)):
        noNeed, f_ext = os.path.splitext(f)
        if f_ext in FEXT:
            my_file_list.append(f)

#                                         (docker run -v) mounted files was empty:
if len(my_file_list) <= 0:
    my_file_list.append('No Data')

#                                         Create and display the widget controls:
flistbx_1 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select Spreadsheet File:'
)
display(flistbx_1)

flistbx_2 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select Dictionary File:'
)
display(flistbx_2)

output_file_button = widgets.Button(
    description='Get Cluster Averages',
    disabled=False,
    button_style='',
    tooltip='get cluster averages button',
    data_file_key='output_file_name'
    )
output_file_button.on_click(get_cluster_averages)
display(output_file_button)


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


# Spreadsheet - select_categorical 

### From a genes x samples spreadsheet and a samples x phenotypes spreadsheet, return both spreadsheets with only the samples corresponding to a category in a phenotype: 
* **Select your genes x samples spreadsheet and samples x phenotypes spreadsheet respectively from the "Select G x S File" and "Select S x P File" dropdown listboxes below, and then select the phenotype id and select category from the next 2 dropdown listboxes.**
* **Press the "Select Categorical" button and the 2 spreadsheets with only the samples corresponding to a category in a phenotype will be written to 2 input files by the same names of those 2 files with "_Slct_Ctg" appended in the same directory of the input files.**

In [11]:
%%html
<style>
div.input {
    display:none;
}
div.output_stderr{
    display:none
}
</style>

In [12]:
#                                         single cell for select, open and select categorical:
#                                         target directory set for docker run -v `pwd`:...   ==  mount user data
target_dir = '../../'

import warnings
warnings.filterwarnings('ignore')

import os
import sys
import pandas as pd
import numpy

from IPython.display import display
import ipywidgets as widgets

#                                         local function to read files and select categorical and write:
def select_categorical(obie_jobie):
    if len(my_file_list) == 0 or my_file_list[0] == 'No Data':
        return
    
    file_name_1 = os.path.join(target_dir, flistbx_1.value)
    file_name_2 = os.path.join(target_dir, flistbx_2.value)
    spreadsheet_df = pd.read_csv(file_name_1, sep='\t', index_col=0, header=0)
    phenotype_df = pd.read_csv(file_name_2, sep='\t', index_col=0, header=0)
    phenotype_id = flistbx_3.value
    select_category = flistbx_4.value
    samples_list = phenotype_df.index[phenotype_df[phenotype_id] == select_category]
    #print(phenotype_df.index)
    # print(phenotype_df[phenotype_id][2])
    # print(samples_list)
    phenotype_category_df = phenotype_df.loc[samples_list]
    spreadsheet_category_df = spreadsheet_df[samples_list]
    
    name_base_1, file_extension_1 = os.path.splitext(file_name_1)
    outfile_name_1 = name_base_1 + '_Slct_Ctg.tsv'
    name_base_2, file_extension_2 = os.path.splitext(file_name_2)
    outfile_name_2 = name_base_2 + '_Slct_Ctg.tsv'
    spreadsheet_category_df.to_csv(outfile_name_1, sep='\t', index=True, header=True)
    phenotype_category_df.to_csv(outfile_name_2, sep='\t', index=True, header=True)
    print('Outputs written to\n', outfile_name_1,'\nand\n',outfile_name_2)

#                                         Get list of (docker run -v) mounted files:
flist = os.listdir(target_dir)
FEXT = ['.tsv', '.txt', '.df']
my_file_list = []
for f in flist:
    if os.path.isfile(os.path.join(target_dir, f)):
        noNeed, f_ext = os.path.splitext(f)
        if f_ext in FEXT:
            my_file_list.append(f)

#                                         (docker run -v) mounted files was empty:
if len(my_file_list) <= 0:
    my_file_list.append('No Data')
    
    
def all_phenotypes(file_rel_path):
    """get all the phenotypes, i.e. column names, of the samples x phenotypes dataframe, 
 which is read from the file_rel_path file """
    try:
        phenotype_df = pd.read_csv(os.path.join(target_dir,file_rel_path), sep='\t', index_col=0, header=0)
        return list(phenotype_df.columns)
    except:
        return ['No Data or Invalid File']


def nan_unique(x):
    """a wrapper of the numpy.unique function that handles the NaN problem, 
    since numpy.unique will return multiple NaN's"""
    a = numpy.unique(x)
    r = []
    has_nan = False
    for i in a:
        if isinstance(i,float) and numpy.isnan(i):
            if has_nan: 
                continue
            else:
                has_nan = True
                r.append(i)
        else:
            r.append(i)
    return numpy.array(r)

def all_categories(file_rel_path,phenotype_id):
    """get all the categories, i.e. the values, of a specfic phenotype, 
    in the dataframe read from file_rel_path file"""
    try:
        phenotype_df = pd.read_csv(os.path.join(target_dir,file_rel_path), sep='\t', index_col=0, header=0)
        # print(list(np.unique(phenotype_df[phenotype_id])))
        return list(nan_unique(phenotype_df[phenotype_id]))
    except:
        return ['No Data or Invalid File']

#                                         Create and display the widget controls:
flistbx_1 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select G x S File:'
)

flistbx_2 = widgets.Dropdown(
    options=my_file_list,
    value=my_file_list[0],
    description='Select S x P File:'
)

flistbx_3 = widgets.Dropdown(
    options=all_phenotypes(my_file_list[0]),
    value=all_phenotypes(my_file_list[0])[0],
    description='Select Phenotype Id:'
)

flistbx_4 = widgets.Dropdown(
    options=all_categories(my_file_list[0],flistbx_3.value),
    value=all_categories(my_file_list[0],flistbx_3.value)[0],
    description='Select Select Category:'
)

def handle_file_change(change):
    """the callback registered to handle changes in the 'value' 
    attribute of widget 'flist_2'"""
    flistbx_3.options = all_phenotypes(change['new']) 
    flistbx_3.value = all_phenotypes(change['new'])[0]
    flistbx_4.options = all_categories(change['new'],flistbx_3.value)
    flistbx_4.value = all_categories(change['new'],flistbx_3.value)[0]

flistbx_2.observe(handle_file_change, names='value')


def handle_phenotype_change(change):
    """the callback registered to handle changes in the 'value' 
    attribute of widget 'flist_3'"""
    flistbx_4.options = all_categories(flistbx_2.value,change['new'])
    flistbx_4.value = all_categories(flistbx_2.value,change['new'])[0]

flistbx_3.observe(handle_phenotype_change, names='value')

display(flistbx_1, flistbx_2, flistbx_3, flistbx_4)

output_file_button = widgets.Button(
    description='Select Categorical',
    disabled=False,
    button_style='',
    tooltip='select categorical button',
    data_file_key='output_file_name'
    )
output_file_button.on_click(select_categorical)
display(output_file_button)


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.
